<img src="https://github.com/gantian127/pymt_roms/blob/master/docs/_static/logo.png?raw=true" width='600' align='center'></a>

## Introduction

[pymt_roms](https://github.com/gantian127/pymt_roms) is a package that uses the [bmi_roms](https://github.com/gantian127/bmi_roms) pacakge to convert [ROMS model](https://www.myroms.org/) datasets into a reusable, plug-and-play data component for PyMT modeling framework developed by Community Surface Dynamics Modeling System ([CSDMS](https://csdms.colorado.edu/wiki/Main_Page)). This allows ROMS datasets to be easily coupled with other datasets or models that expose a Basic Model Interface.

The current implementation supports 2D, 3D and 4D ROMS output datasets defined with geospatial and/or time dimensions (e.g., dataset defined with dimensions as [time, s_rho, eta_rho, xi_rho])

To install pymt_roms, please follow the instructions [here](https://github.com/gantian127/pymt_roms/#pymt_roms).

**Suggested citation**: Gan, T. (2023). Jupyter Notebooks for the ROMS Data Component, HydroShare, https://www.hydroshare.org/resource/5bed8401cfe04c38b0f84119b1999482/

## Coding Examples

This section provides an example to demonstrate how to use the pymt_roms package to access and load the ROMS dataset for visualization.

Import Roms class and instantiate it. A configuration file (yaml file) is required to provide the parameter settings. An example config_file.yaml file is provided in the same folder with this Jupyter Notebook. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from pymt.models import Roms

# initiate a data component
data_comp = Roms()
data_comp.initialize('config_file.yaml')

If you check the content in the config_file.yaml, you will find that the parameter includes a file name to load the local ROMS model dataset. The example.nc file is provided in the same folder with this Jupyter Notebook. You can also set the "filename" value with an OPeNDAP Data URL to access remote dataset and donwload it by setting the "download" parameter as True. Please see more details [here](https://bmi-roms.readthedocs.io/en/latest/?badge=latest#parameters) for the parameter settings.

In [ ]:
!cat config_file.yaml

Use variable related methods to check the variable information of the ROMS dataset.

In [ ]:
# get variable info
for var_name in data_comp.output_var_names:
    var_unit = data_comp.var_units(var_name)
    var_location = data_comp.var_location(var_name)
    var_type = data_comp.var_type(var_name)
    var_grid = data_comp.var_grid(var_name)
    var_itemsize = data_comp.var_itemsize(var_name)
    var_nbytes = data_comp.var_nbytes(var_name)

    print('variable_name: {} \nvar_unit: {} \nvar_location: {} \nvar_type: {} \nvar_grid: {} \nvar_itemsize: {}'
        '\nvar_nbytes: {} \n'. format(var_name, var_unit, var_location, var_type, var_grid, var_itemsize, var_nbytes))

Use time related methods to check the time information of the ROMS dataset. The time values are stored in a format which follows [CF convention](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.8/cf-conventions.pdf).

In [ ]:
# get time info
start_time = data_comp.start_time
end_time = data_comp.end_time
time_step = data_comp.time_step
time_units = data_comp.time_units
time_steps = int((end_time - start_time)/time_step) + 1

print('start_time: {} \nend_time: {} \ntime_step: {} \ntime_units: {} \ntime_steps: {}'.format(
        start_time, end_time, time_step, time_units, time_steps))

Use grid related methods to check the grid information of the ROMS dataset.  You will find out that the variables are defined on different grids.

In [ ]:
# get variable grid info
for var_name in data_comp.output_var_names:
    grid_id = data_comp.var_grid(var_name)
    grid_type = data_comp.grid_type(grid_id)
    grid_rank = data_comp.grid_ndim(grid_id)
    grid_shape = data_comp.grid_shape(grid_id)
    grid_spacing = data_comp.grid_spacing(grid_id)
    grid_origin = data_comp.grid_origin(grid_id)

    print('var_name: {} \ngrid_id: {} \ngrid_type: {} \ngrid_rank: {} \ngrid_shape: {} \ngrid_spacing: {} \ngrid_origin: {} \n'.format(
        var_name, grid_id, grid_type, grid_rank, grid_shape, grid_spacing, grid_origin))

Use get_value( ) method to get the variable data as a numpy array. Please note that the default behavior of pymt components is to flatten data arrays. So make a new variable that restores the dimensionality of the data. 

The code below shows how to get the variable values for the 'time-averaged salinity' and make a contour plot. The latitude and longitude values can be obtained from the 'latitude of RHO-points' and 'longitude of RHO-points' variables.

In [ ]:
# get variable data
data = data_comp.get_value('time-averaged salinity')
data_3D = data.reshape([40, 106, 242]) 

# get lon and lat data
lat = data_comp.get_value('latitude of RHO-points').reshape([106, 242])
lon = data_comp.get_value('longitude of RHO-points').reshape([106, 242])

In [ ]:
# make a contour plot
fig = plt.figure(figsize=(10,7))
im = plt.contourf(lon, lat, data_3D[0], levels=36)
fig.colorbar(im)
plt.axis('equal')
plt.xlabel('Longitude [degree_east]')
plt.ylabel('Latitude [degree_north]')
plt.title('ROMS model data of time-averaged salinity')


Complete the example by finalizing the component. finalize( ) method performs tasks that take place after using the data component, such as deallocating memory and closing files.

In [ ]:
data_comp.finalize()